# EOP Visualization

In [3]:
import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.graph_objects as go

import sys
import os

# Dynamically get the absolute path to the repo root
current_file = os.path.abspath("")
#print(current_file)
project_folder_path = os.path.abspath(os.path.join(current_file, "..", ".."))
#print(project_folder_path)

# Add the project folder to Python’s module search path to enable importing custom project modules
if project_folder_path not in sys.path:
    sys.path.insert(0, project_folder_path)

from AttitudeKinematicsLib.EulerAngles import *

In [2]:
from IPython.core.magic import register_cell_magic

@register_cell_magic
def skip(line, cell):
    """
    A custom cell magic to skip execution of the cell.
    """
    return  # Does nothing

## 1 - Parsing and Understanding EOP file from Celestrak

In [5]:
eop_df = pd.read_csv("EOP-All.csv")
eop_df

,DATE,MJD,X,Y,UT1-UTC,LOD,DPSI,DEPS,DX,DY,DAT,DATA_TYPE
0,1962-01-01,37665,-0.012700,0.213000,0.032634,0.001723,0.064261,0.006067,0.000000,0.000000,2,O
1,1962-01-02,37666,-0.015900,0.214100,0.032055,0.001669,0.063979,0.006290,0.000000,0.000000,2,O
2,1962-01-03,37667,-0.019000,0.215200,0.031553,0.001582,0.063870,0.006515,0.000000,0.000000,2,O
3,1962-01-04,37668,-0.021999,0.216301,0.031144,0.001496,0.063893,0.006574,0.000000,0.000000,2,O
4,1962-01-05,37669,-0.024799,0.217301,0.030815,0.001416,0.063880,0.006374,0.000000,0.000000,2,O
...,...,...,...,...,...,...,...,...,...,...,...,...
23396,2026-01-21,61061,0.066155,0.335907,0.079663,0.000282,-0.113959,-0.007335,0.000370,-0.000179,37,P
23397,2026-01-22,61062,0.065212,0.336803,0.079301,0.000486,-0.113961,-0.007421,0.000359,-0.000188,37,P
23398,2026-01-23,61063,0.064377,0.337725,0.078770,0.000614,-0.114086,-0.007501,0.000346,-0.000193,37,P
23399,2026-01-24,61064,0.063581,0.338595,0.078162,0.000632,-0.114236,-0.007514,0.000336,-0.000191,37,P


Earth Orientation Parameters (EOP) describe how the Earth’s rotation and orientation in space vary over time. These parameters are essential for accurately transforming between the Earth-Centered Inertial (ECI) and Earth-Centered Earth-Fixed (ECEF) reference frames.

| Column     | Description                                                       | Units           | Usage in Frame Transformation                        |
|------------|-------------------------------------------------------------------|------------------|------------------------------------------------------|
| `Date`     | Date of EOP entry (typically in Modified Julian Date, MJD)        | —               | Time-tag of each row                                |
| `X`        | Polar motion along Earth-fixed x-axis ($x_p$)                     | arcseconds       | Rotation about x-axis due to polar wobble           |
| `Y`        | Polar motion along Earth-fixed y-axis ($y_p$)                     | arcseconds       | Rotation about y-axis due to polar wobble           |
| `UT1-UTC`  | Difference between UT1 and UTC                                     | seconds          | Used to compute Earth Rotation Angle (ERA)          |
| `LOD`      | Excess Length of Day compared to mean 86400s                      | milliseconds     | Reflects changes in Earth's rotation rate           |
| `DPSI`     | Correction to nutation in longitude ($\Delta\psi$)               | arcseconds       | Correction to IAU precession-nutation models        |
| `DEPS`     | Correction to nutation in obliquity ($\Delta\epsilon$)           | arcseconds       | Correction to IAU precession-nutation models        |
| `DX`       | Correction to celestial pole x-offset                             | milliarcseconds  | Alternative to $\Delta\psi$ for IAU 2006+ models     |
| `DY`       | Correction to celestial pole y-offset                             | milliarcseconds  | Alternative to $\Delta\epsilon$ for IAU 2006+ models |

**<ins>The units of Arcseconds and Milliarcseconds</ins>**

 **Arcsecond (″)** is a unit of angular measurement:

  $$
  1^\circ = 60' = 3600'' \quad (\text{arcseconds})
  $$

  $$
  1'' = \frac{\pi}{180 \times 3600} \approx 4.848 \times 10^{-6} \, \text{radians}
  $$

- **Milliarcsecond (mas)**:

  $$
  1\, \text{mas} = \frac{1}{1000}'' \approx 4.848 \times 10^{-9} \, \text{radians}
  $$

Always convert angular units to **radians** before using them in rotation matrices.


**<ins>Necessity of EOPs</ins>**

To transform a position from **ECI** to **ECEF**, apply the following rotations:

1. **Precession & Nutation**  
   Modeled by IAU theories, with corrections from $\Delta\psi$, $\Delta\epsilon$ or $dx$, $dy$

2. **Earth Rotation Angle (ERA)**  
   Derived from UT1 to rotate about the Z-axis:

   $$
   \theta = 2\pi \left(0.7790572732640 + 1.00273781191135448 \cdot (JD_{UT1} - 2451545.0)\right) \mod 2\pi
   $$

3. **Polar Motion**  
   Corrected by applying:

   $$
   \mathbf{W}_{\text{polar}} = R_x(y_p) \cdot R_y(x_p)
   $$

The full transformation sequence:

$$
\text{ECI} \xrightarrow{\text{Precession + Nutation}} \text{CIRF} \xrightarrow{\text{Earth Rotation}} \text{TIRF} \xrightarrow{\text{Polar Motion}} \text{ECEF}
$$
